In [2]:
from google.colab import files
files.upload()

Saving signdata.zip to signdata.zip


{'signdata.zip': b'PK\x03\x04\x14\x00\x00\x00\x08\x00 \x89\xa8N\xc3\xe2rD\x13r\x00\x00}\xd1\x01\x00&\x00\x00\x00signdata/PHOENIX-2014-T.dev.corpus.csv\xad\xbd\xcbr\x1bI\x96&\xbc\xc7S`=m\x9aA\xe0\x8e%T\x82$\x96x\x91\x01d\xaaZ\x9b2P\x0c\x92H\x82\x80\x06\x01\x88\x7f\xc9\xe2]j\x93\xcf\x90\xab\xdc\xf1\xc5\xfesu?\x1e\x08G\xb8\xaa\xc7\xac\xbb\x92\x02\\\xa2_\xcf\xf5;\xdf\xd9,\x9f\xf3\xf2\xc7\xea.\xdf\x96\xc5~\xb9\xdb\x97\xf9\xe6\xae,\xbe\xe7\xcb\xa7|Wnw\xfb\xc7r\xbf[n\x8a\xf5r\xbf\xdanZY6=<\x1c\x8a\xfd?\xbb\x9d\xac\xf3\xcf/\xf9\xdd&/\xee\x96\xff\xfa\xe7~\xf9\x90\x17\xc5\xb7\xc7\xe5\xe1M\xb7L\x19\xf4\x7f\xb2\xff\xf3\xbf\xfe\xf7\xf7\xcdC\xf9&\xc3\xff[\xac\x1e6\xf9\xae3.\xdf\xcdo\xfe\xf6\xa9}}6{\xdf\xfetuq1\xbb,\xf7\xab\xfc>\xdf\xb5\xd7\x87\xfb\xfd\xdd\xee\xf0\xed\xa9}\x9b\x17\xfb\xd5\xf3\xf3\xbe\xbd\xda\xb4\xef\xf2M{\xf3\xfa\xc7\xb7\xc7b\x0f?\xe1/\xd8\xb4\x0f\x9b\x02\xc6\xbf\xe4\xfb}\xbeK\x99p/e\xc2\xbd\xd8\x84g\x8b7og\xeff7\xd7\xb3\xeb\xf6og\xb3\xf3\xf6\x97\xab\xf3O\xb3\xf6\xcd\xe5;X\xc0\xec\xf

In [3]:
!unzip signdata.zip

Archive:  signdata.zip
  inflating: signdata/PHOENIX-2014-T.dev.corpus.csv  
  inflating: signdata/PHOENIX-2014-T.test.corpus.csv  
  inflating: signdata/PHOENIX-2014-T.train.corpus.csv  


In [1]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf

tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time

print(tf.__version__)

1.13.1


In [2]:
import pandas as pd
train=pd.read_csv('/content/signdata/PHOENIX-2014-T.train.corpus.csv',sep='|')
test=pd.read_csv('/content/signdata/PHOENIX-2014-T.test.corpus.csv',sep='|')
dev=pd.read_csv('/content/signdata/PHOENIX-2014-T.dev.corpus.csv',sep='|')
dev.columns

Index(['name', 'video', 'start', 'end', 'speaker', 'orth', 'translation'], dtype='object')

In [0]:
gertrain=train['translation'].tolist()
glosstrain=train['orth'].tolist()

gertest=test['translation'].tolist()
glosstest=test['orth'].tolist()

gerdev=dev['translation'].tolist()
glossdev=dev['orth'].tolist()

In [0]:
#data.head()

In [0]:
#len(german[1])

In [0]:
german=[]
gloss=[]
for i in range(len(gertrain)):
  german.append(gertrain[i])
  gloss.append(glosstrain[i])

for i in range(len(gertest)):
  german.append(gertest[i])
  gloss.append(glosstest[i])

for i in range(len(gerdev)):
  german.append(gerdev[i])
  gloss.append(glossdev[i])

In [7]:
len(gertrain)+len(gertest)+len(gerdev)

8257

In [8]:
len(german)

8257

In [0]:
original_word_pairs=[[gloss[i],german[i]] for i in range(len(gloss))]

data = pd.DataFrame(original_word_pairs, columns=["eng", "es"])

In [70]:
data.head()

,eng,es
0,JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR,und nun die wettervorhersage für morgen donner...
1,ORT REGEN DURCH REGEN KOENNEN UEBERSCHWEMMUNG ...,mancherorts regnet es auch länger und ergiebig...
2,NORDWEST HEUTE NACHT TROCKEN BLEIBEN SUEDWEST ...,im nordwesten bleibt es heute nacht meist troc...
3,TAGSUEBER OFT REGEN GEWITTER KOENNEN MANCHMAL ...,auch am tag gibt es verbreitet zum teil kräfti...
4,WOLKE LOCH SPEZIELL NORDWEST,größere wolkenlücken finden sich vor allem im ...


In [71]:
data.shape

(8257, 2)

In [0]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    """
    Normalizes latin chars with accent to their canonical decomposition
    """
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [73]:
# Now we do the preprocessing using pandas and lambdas
data["eng"] = data.eng.apply(lambda w: preprocess_sentence(w))
data["es"] = data.es.apply(lambda w: preprocess_sentence(w))
data.sample(10)


,eng,es
6183,<start> region <end>,<start> sonst regnet es gebietsweise <end>
6011,<start> dienstag teilweise freundlich alpen ix...,<start> am dienstag ist es teilweise freundlic...
3232,<start> in kommend bisschen kuehl wind bissche...,<start> die nachsten tage werden etwas kuhler ...
7698,<start> fluss ungefaehr fuenfzehn ix siebzehn ...,<start> am rhein werden es dann wenigstens mal...
1703,<start> luft kuehl wehen ankommen deutschland ...,<start> dabei gelangt deutlich kuhlere luft au...
456,<start> suedost grund regen muessen mit rechne...,<start> wegen des regens muss heute nacht im s...
675,<start> morgen bleiben wolke kommen koennen sc...,<start> die halten sich zum teil auch morgen u...
7224,<start> naechste woche kuehl abwechseln wetter...,<start> in der neuen woche kuhlt es dann bei w...
2313,<start> freitag sonne wolke teil schauer gewit...,<start> am freitag mal sonne mal wolken teilwe...
4321,<start> ix ost mehr stark berg bayern ix regen...,<start> im sudosten ist es heute nacht meist s...


In [0]:
data.columns
testgerman=data['eng'].tolist()
testgloss=data['es'].tolist()


In [75]:
len(testgerman)

8257

In [0]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
    def __init__(self, lang):
        """ lang are the list of phrases from each language"""
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        
        self.create_index()
        
    def create_index(self):
        for phrase in self.lang:
            # update with individual tokens
            self.vocab.update(phrase.split(' '))
            
        # sort the vocab
        self.vocab = sorted(self.vocab)

        # add a padding token with index 0
        self.word2idx['<pad>'] = 0
        
        # word to index mapping
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1 # +1 because of pad token
        
        # index to word mapping
        for word, index in self.word2idx.items():
            self.idx2word[index] = word        

In [77]:
# index language using the class above
inp_lang = LanguageIndex(data["es"].values.tolist())
targ_lang = LanguageIndex(data["eng"].values.tolist())
# Vectorize the input and target languages
input_tensor = [[inp_lang.word2idx[s] for s in es.split(' ')]  for es in data["es"].values.tolist()]
target_tensor = [[targ_lang.word2idx[s] for s in eng.split(' ')]  for eng in data["eng"].values.tolist()]
input_tensor[:10]

[[2, 2523, 1734, 475, 2815, 855, 1557, 491, 442, 2976, 199, 1],
 [2,
  1471,
  1901,
  712,
  163,
  1382,
  2523,
  674,
  163,
  1436,
  2498,
  2125,
  2822,
  1548,
  1],
 [2,
  1194,
  1717,
  346,
  712,
  1105,
  1587,
  1490,
  2442,
  2190,
  1568,
  1514,
  2374,
  1321,
  1980,
  931,
  2773,
  1768,
  1514,
  350,
  2523,
  489,
  1],
 [2,
  163,
  92,
  2347,
  987,
  712,
  2579,
  2937,
  2370,
  1319,
  1978,
  1750,
  962,
  2523,
  1197,
  1469,
  1896,
  728,
  675,
  1880,
  1],
 [2, 1019, 663, 2878, 750, 2106, 2661, 66, 1194, 1717, 1],
 [2, 1194, 625, 1105, 1587, 1735, 1633, 92, 1747, 335, 2121, 1006, 1],
 [2,
  1557,
  56,
  2377,
  2821,
  1105,
  1638,
  335,
  849,
  1197,
  446,
  1396,
  335,
  2118,
  1006,
  1],
 [2,
  92,
  790,
  1254,
  712,
  1197,
  446,
  1782,
  2374,
  1382,
  2523,
  1318,
  1898,
  2681,
  1161,
  1],
 [2, 2190, 2725, 1514, 1980, 2523, 969, 475, 2541, 163, 92, 2864, 261, 1],
 [2, 92, 2381, 106, 2106, 2764, 1]]

In [78]:
target_tensor[:10]

[[2, 408, 917, 526, 182, 974, 255, 1],
 [2, 593, 629, 199, 629, 432, 813, 432, 1],
 [2, 572, 367, 536, 798, 126, 771, 432, 629, 593, 315, 165, 1],
 [2, 778, 587, 629, 315, 432, 486, 629, 859, 629, 1],
 [2, 938, 474, 738, 572, 1],
 [2, 267, 367, 536, 694, 267, 717, 327, 1],
 [2, 526, 788, 322, 919, 367, 553, 120, 286, 967, 451, 402, 714, 967, 327, 1],
 [2, 275, 594, 432, 629, 746, 629, 878, 813, 432, 1],
 [2, 630, 898, 629, 315, 53, 935, 1],
 [2, 788, 126, 322, 1]]

In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
# calculate the max_length of input and output tensor
max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)

In [81]:
print(max_length_inp, max_length_tar)

54 34


In [0]:
def pad_sequences(x, max_len):
    padded = np.zeros((max_len), dtype=np.int64)
    if len(x) > max_len: padded[:] = x[:max_len]
    else: padded[:len(x)] = x
    return padded

In [83]:
# inplace padding
input_tensor = [pad_sequences(x, max_length_inp) for x in input_tensor]
target_tensor = [pad_sequences(x, max_length_tar) for x in target_tensor]
len(target_tensor)

8257

In [84]:
len(gertrain)

7096

In [85]:
len(gertest)+len(gertrain)

7738

In [0]:
#len(input_tensor)
input_tensor_train=input_tensor[0:len(gertrain)]
input_tensor_val=input_tensor[len(gertest)+len(gertrain):len(gertrain)+len(gertest)+len(gerdev)]
target_tensor_train=target_tensor[0:len(gertrain)]
target_tensor_val=target_tensor[len(gertest)+len(gertrain) : len(gertrain)+len(gertest)+len(gerdev)]

In [87]:
# Creating training and validation sets using an 80-20 split
#input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(7096, 7096, 519, 519)

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [0]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
  if tf.test.is_gpu_available():
    return tf.keras.layers.CuDNNGRU(units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform')
  else:
    return tf.keras.layers.GRU(units, 
                               return_sequences=True, 
                               return_state=True, 
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform')

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [0]:
optimizer = tf.train.AdamOptimizer()


def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [0]:
loss=[]
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    #loss.append([total_loss / N_BATCH])
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.8284


In [0]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word2idx[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [0]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    
    fontdict = {'fontsize': 14}
    
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()

In [0]:
def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence, attention_plot = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    #print('Input: {}'.format(sentence))
    #print('Predicted translation: {}'.format(result))
    
    #attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    #plot_attention(attention_plot, sentence.split(' '), result.split(' '))
    return(result)

In [0]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
te=pd.read_csv('/content/signdata/PHOENIX-2014-T.test.corpus.csv',sep='|')
ge=te['translation'].tolist()
go=te['orth'].tolist()

In [0]:
len(go)

642

In [0]:
ger[1]

'mancherorts regnet es auch länger und ergiebig auch lokale überschwemmungen sind wieder möglich'

In [0]:
pred=translate(go[1], encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ=20)

In [0]:
pred[0:-1]

'am donnerstag regnet es im norden und nordwesten mal sonne mal wieder sonst gibt es mal sonne mal wieder sonst'

In [0]:
from nltk.translate.bleu_score import sentence_bleu

In [0]:
pred=[]
bleu=[]
no=[]

In [0]:
pred=[]
bleu1=[]
bleu2=[]
bleu3=[]
bleu4=[]
no=[]

In [0]:
import csv

In [0]:
len(train)
len(test)

642

In [0]:

for i in range(len(go)):
  #no.append(i)
  pred=translate(go[i], encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ=34)
  #pred.append(translate(go[i], encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ=20))
  # cumulative BLEU scores
  #print(pred)
  pred1=pred.split(' ')
  reference = [ pred1[0:-2] ]
  print(reference)
  candidate = ge[i].split(' ')
  print(candidate)
  bleu1=(sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
  bleu2=(sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0)))
  bleu3=(sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0)))
  bleu4=(sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25)))
  #print('Cumulative 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
  #print('Cumulative 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0)))
  #print('Cumulative 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0)))
  #print('Cumulative 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25)))
  row=[go[i],ge[i],pred,bleu1,bleu2,bleu3,bleu4]
  with open('nmttest.csv', 'a') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(row)

  csvFile.close()
  print(i," is done")

[['regen', 'der', 'regen', 'der', 'fall', 'sonst', 'ist', 'es', 'regen', 'im', 'norden', 'aus', 'und', 'da', 'lockert', 'es', 'mitunter', 'noch', 'gebietsweise', 'auf', 'und', 'da', 'lockert', 'es', 'mitunter', 'noch', 'gebietsweise', 'auf', 'und', 'da', 'lockert', 'es', 'mitunter']]
['regen', 'und', 'schnee', 'lassen', 'an', 'den', 'alpen', 'in', 'der', 'nacht', 'nach', 'im', 'norden', 'und', 'nordosten', 'fallen', 'hier', 'und', 'da', 'schauer', 'sonst', 'ist', 'das', 'klar']
0  is done


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


[['am', 'donnerstag', 'regnet', 'es', 'im', 'norden', 'und', 'nordwesten', 'mal', 'sonne', 'mal', 'wieder', 'sonst', 'gibt', 'es', 'mal', 'sonne', 'mal', 'wieder', 'sonst', 'gibt', 'es', 'mal', 'sonne', 'mal', 'wieder', 'sonst', 'gibt', 'es', 'mal', 'sonne', 'mal', 'wieder']]
['am', 'donnerstag', 'regen', 'in', 'der', 'nordhälfte', 'in', 'der', 'südhälfte', 'mal', 'sonne', 'mal', 'wolken', 'ähnliches', 'wetter', 'dann', 'auch', 'am', 'freitag']
1  is done
[['dabei', 'wird', 'es', 'dann', 'auch', 'noch', 'schnee', 'und', 'regen']]
['vom', 'nordmeer', 'zieht', 'ein', 'kräftiges', 'tief', 'heran', 'und', 'bringt', 'uns', 'ab', 'den', 'morgenstunden', 'heftige', 'schneefälle', 'zum', 'teil', 'auch', 'gefrierenden', 'regen']
2  is done


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


[['am', 'wochenende', 'wird', 'es', 'fur', 'alle', 'am', 'samstag', 'fur', 'alle', 'am', 'samstag', 'fur', 'alle', 'am', 'samstag', 'fur', 'alle', 'am', 'samstag', 'fur', 'alle', 'am', 'samstag', 'fur', 'alle', 'am', 'samstag', 'fur', 'alle', 'am', 'samstag', 'fur']]
['sonnig', 'geht', 'es', 'auch', 'ins', 'wochenende', 'samstag', 'ein', 'herrlicher', 'tag', 'mit', 'temperaturen', 'bis', 'siebzehn', 'grad', 'hier', 'im', 'westen']
3  is done
[['ein', 'hoch', 'morgen', 'noch', 'fur', 'morgen', 'ein', 'hoch', 'morgen', 'noch', 'fur', 'morgen', 'ein', 'hoch', 'morgen', 'noch', 'fur', 'morgen', 'ein', 'hoch', 'morgen', 'noch', 'fur', 'morgen', 'ein', 'hoch', 'morgen', 'noch', 'fur', 'morgen', 'ein', 'hoch', 'morgen']]
['deutschland', 'liegt', 'morgen', 'unter', 'hochdruckeinfluss', 'der', 'die', 'wolken', 'weitgehend', 'vertreibt']
4  is done
[['am', 'sonntag', 'uberwiegen', 'die', 'wolken', 'teilweise', 'unwetterartige', 'schauer', 'oder', 'gewitter', 'die', 'vor', 'allem', 'im', 'nordwes

In [0]:
len(pred)

14

In [38]:
from google.colab import files
files.upload()

Saving glosstogerman50epoches.csv to glosstogerman50epoches.csv


{'glosstogerman50epoches.csv': b'gloss,german,predicted german ,bleu1,bleu2,bleu3,bleu4\r\nREGEN SCHNEE REGION VERSCHWINDEN NORD REGEN KOENNEN REGION STERN KOENNEN SEHEN,regen und schnee lassen an den alpen in der nacht nach im norden und nordosten fallen hier und da schauer sonst ist das klar,regen der regen der fall sonst ist es regen im norden aus und da lockert es mitunter noch gebietsweise auf und da lockert es mitunter noch gebietsweise auf und da lockert es mitunter noch ,0.286370533,0.160225139,0.262875243,0.331844874\r\nDONNERSTAG NORDWEST REGEN REGION SONNE WOLKE WECHSELHAFT DANN FREITAG AEHNLICH WETTER,am donnerstag regen in der nordh\xc3\xa4lfte in der s\xc3\xbcdh\xc3\xa4lfte mal sonne mal wolken \xc3\xa4hnliches wetter dann auch am freitag,am donnerstag regnet es im norden und nordwesten mal sonne mal wieder sonst gibt es mal sonne mal wieder sonst gibt es mal sonne mal wieder sonst gibt es mal sonne mal wieder sonst ,0.125953414,0.100236551,0.066961766,0.107869235\r\nKRAE

In [0]:
import pandas as pd
nmt=pd.read_csv('/content/glosstogerman50epoches.csv')
#tdata.head()

In [46]:
nmt.columns

Index(['gloss', 'german', 'predicted german ', 'bleu1', 'bleu2', 'bleu3',
       'bleu4'],
      dtype='object')

In [0]:
txt=nmt[nmt.columns[1]].tolist()
pred=nmt[nmt.columns[2]].tolist()
b1=nmt[nmt.columns[3]].tolist()
b2=nmt[nmt.columns[4]].tolist()
b3=nmt[nmt.columns[5]].tolist()
b4=nmt[nmt.columns[6]].tolist()

In [48]:
def Nmaxelements(list1, N): 
    final_list = [] 
  
    for i in range(0, N):  
        max1 = 0
          
        for j in range(len(list1)):      
            if list1[j] > max1: 
                max1 = list1[j]; 
                  
        list1.remove(max1); 
        final_list.append(max1) 
          
    print(final_list)
    return(final_list)
# Calling the function 
N=60
bs1=Nmaxelements(b1, N) 
bs2=Nmaxelements(b2, N) 
bs3=Nmaxelements(b3, N) 
bs4=Nmaxelements(b4, N) 



[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.8824969029999999, 0.833333333, 0.833333333, 0.833333333, 0.818181818, 0.818181818, 0.818181818, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.8366600270000001, 0.81703337, 0.809039835, 0.774596669, 0.756787469, 0.737533758, 0

In [0]:
b1=nmt[nmt.columns[3]].tolist()
b2=nmt[nmt.columns[4]].tolist()
b3=nmt[nmt.columns[5]].tolist()
b4=nmt[nmt.columns[6]].tolist()

In [0]:
bd1=[]
bd2=[]
bd3=[]
bd4=[]
for i in range(len(bs1)):
  #print(bs1[i])
  bd1.append(b1.index(bs1[i]))
  bd2.append(b2.index(bs2[i]))
  bd3.append(b3.index(bs3[i]))
  bd4.append(b4.index(bs4[i]))
  

In [0]:
bl1 = list(set(bd1))
bl2 = list(set(bd2))
bl3 = list(set(bd3))
bl4 = list(set(bd4))

In [54]:
for i in bl1:
  print('original:',txt[i])
  print("predicted:",pred[i])
  print("___________________________________________________")
print("=====================================================================")
print("=====================================================================")
for i in bl2:
  print('original:',txt[i])
  print("predicted:",pred[i])
  print("___________________________________________________")
print("=====================================================================")
print("=====================================================================")

for i in bl3:
  print('original:',txt[i])
  print("predicted:",pred[i])
  print("___________________________________________________")
print("=====================================================================")
print("=====================================================================")

for i in bl4:
  print('original:',txt[i])
  print("predicted:",pred[i])
  print("___________________________________________________")
print("=====================================================================")
print("=====================================================================")


original: guten abend liebe zuschauer
predicted: guten abend liebe zuschauer <end> 
___________________________________________________
original: in der nacht sinken die temperaturen auf vierzehn bis sieben grad
predicted: heute nacht sinken die temperaturen auf vierzehn bis sieben grad <end> 
___________________________________________________
original: und nun die wettervorhersage für morgen freitag den fünfzehnten oktober
predicted: und nun die wettervorhersage fur morgen freitag den funfzehnten oktober <end> 
___________________________________________________
original: und nun die wettervorhersage für morgen dienstag den ersten februar
predicted: und nun die wettervorhersage fur morgen dienstag den ersten februar <end> 
___________________________________________________
original: der wind weht meist schwach aus unterschiedlichen richtungen
predicted: der wind weht meist nur schwach aus unterschiedlichen richtungen <end> 
___________________________________________________
origina

In [55]:
!pip install googletrans

In [0]:
from googletrans import Translator
translator = Translator()


In [0]:
btt1=translator.translate([txt[i] for i in bl1], dest='en')
bpt1=translator.translate([pred[i] for i in bl1], dest='en')


btt2=translator.translate([txt[i] for i in bl2], dest='en')
bpt2=translator.translate([pred[i] for i in bl2], dest='en')

btt3=translator.translate([txt[i] for i in bl3], dest='en')
bpt3=translator.translate([pred[i] for i in bl3], dest='en')


btt4=translator.translate([txt[i] for i in bl4], dest='en')
bpt4=translator.translate([pred[i] for i in bl4], dest='en')


In [62]:
for translation,translations in zip(btt1,bpt1):
  #for  in bpt1:
  print(translation.origin)
  print(translations.origin)
  print( translation.text)
  print( translations.text)
  print(" ____________________________________________________________________________________________ ")

guten abend liebe zuschauer
guten abend liebe zuschauer <end> 
good evening dear spectators
good evening dear audience <end>
 ____________________________________________________________________________________________ 
in der nacht sinken die temperaturen auf vierzehn bis sieben grad
heute nacht sinken die temperaturen auf vierzehn bis sieben grad <end> 
at night temperatures drop to fourteen to seven degrees
Tonight, temperatures will drop to fourteen to seven degrees <end>
 ____________________________________________________________________________________________ 
und nun die wettervorhersage für morgen freitag den fünfzehnten oktober
und nun die wettervorhersage fur morgen freitag den funfzehnten oktober <end> 
and now the weather forecast for tomorrow friday the fifteenth october
and now the weather forecast for tomorrow friday the fifteenth october <end>
 ____________________________________________________________________________________________ 
und nun die wettervorhersage f

In [63]:
for translation,translations in zip(btt2,bpt2):
  #for  in bpt1:
  print(translation.origin)
  print(translations.origin)
  print( translation.text)
  print( translations.text)
  print(" ____________________________________________________________________________________________ ")

im nordosten bleibt es meist trocken
im nordosten bleibt es trocken <end> 
in the northeast it stays mostly dry
in the northeast it stays dry <end>
 ____________________________________________________________________________________________ 
teilweise ist es auch klar
teilweise auch sonst teils klar <end> 
partly it is also clear
sometimes also partly clear <end>
 ____________________________________________________________________________________________ 
guten abend liebe zuschauer
guten abend liebe zuschauer <end> 
good evening dear spectators
good evening dear audience <end>
 ____________________________________________________________________________________________ 
und nun die wettervorhersage für morgen donnerstag den dritten märz
und nun die wettervorhersage fur morgen donnerstag den dritten marz <end> 
and now the weather forecast for tomorrow thursday the third march
and now the weather forecast for tomorrow thursday the third march <end>
 __________________________________

In [64]:
for translation,translations in zip(btt3,bpt3):
  #for  in bpt1:
  print(translation.origin)
  print(translations.origin)
  print( translation.text)
  print( translations.text)
  print(" ____________________________________________________________________________________________ ")

teilweise ist es auch klar
teilweise auch sonst teils klar <end> 
partly it is also clear
sometimes also partly clear <end>
 ____________________________________________________________________________________________ 
guten abend liebe zuschauer
guten abend liebe zuschauer <end> 
good evening dear spectators
good evening dear audience <end>
 ____________________________________________________________________________________________ 
im süden bleibt es morgen unter hochdruckeinfluss zunächst noch recht freundlich und warm
im suden halt sich morgen noch hochsommerlich warm <end> 
in the south, it will remain tomorrow at first under the influence of high pressure, still quite friendly and warm
in the south, tomorrow still warm summer <end>
 ____________________________________________________________________________________________ 
im süden entladen sich später hier und da heftige gewitter
im suden entwickeln sich noch einzelne gewitter <end> 
Later in the south, heavy thunderstorms ar

In [65]:
for translation,translations in zip(btt4,bpt4):
  #for  in bpt1:
  print(translation.origin)
  print(translations.origin)
  print( translation.text)
  print( translations.text)
  print(" ____________________________________________________________________________________________ ")

das hoch gewinnt wieder an kraft und beschert uns am wochenende viel sonnenschein bei meist sommerlichen temperaturen
au erdem lenkt das ist eine deutliche abkuhlung sorgt bei uns fur steigende temperaturen <end> 
the high regains its strength and gives us a lot of sunshine during the summer, with mostly summery temperatures
In addition, this is a significant cooling causes us for rising temperatures <end>
 ____________________________________________________________________________________________ 
dazu weht ein schwacher bis mäßiger wind aus südost bis süd
der wind weht schwach bis ma ig im nordosten aus unterschiedlichen richtungen <end> 
In addition a weak to moderate wind blows from southeast to south
the wind blows weakly to moderately in the northeast from different directions <end>
 ____________________________________________________________________________________________ 
örtlich schauer oder gewitter die heftig sein können
hier und da schauer und gewitter <end> 
local showe